In [57]:
from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset, load_metric
from datasets import DatasetDict
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/maxhager/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(42)


### Model


In [59]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparam):
        super(T5FineTuner, self).__init__()
        self.hparam = hparam

        self.model = T5ForConditionalGeneration.from_pretrained(
            hparam.model_name_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(
            hparam.model_name_or_path
        )
        self.save_hyperparameters()

    def is_logger(self):
        return True

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparam.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparam.learning_rate, eps=self.hparam.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self,
                       epoch=None,
                       batch_idx=None,
                       optimizer=None,
                       optimizer_idx=None,
                       optimizer_closure=None,
                       on_tpu=None,
                       using_native_amp=None,
                       using_lbfgs=None
                       ):

        optimizer.step(closure=optimizer_closure)
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(
            self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="train", args=self.hparam)
        dataloader = DataLoader(train_dataset, batch_size=self.hparam.train_batch_size,
                                drop_last=True, shuffle=True, num_workers=2)
        t_total = (
            (len(dataloader.dataset) //
             (self.hparam.train_batch_size * max(1, self.hparam.n_gpu)))
            // self.hparam.gradient_accumulation_steps
            * float(self.hparam.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparam.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(
            tokenizer=self.tokenizer, type_path="validation", args=self.hparam)
        return DataLoader(val_dataset, batch_size=self.hparam.eval_batch_size, num_workers=2)


In [60]:
logger = logging.getLogger(__name__)


class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(
                pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(
                            key, str(metrics[key])))


In [61]:
args_dict = dict(
    data_dir="jnlpba",  # path for data files
    output_dir="",  # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,  # todo figure out
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    # fp_16=True, # if you want to enable 16-bit training then install apex and set this to true
    # opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    # max_grad_norm=1, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)


### Dataset


In [80]:
from datasets import DatasetDict, Dataset

jnlpba = load_dataset('jnlpba', split=['train[:10]', "validation[:10]"])

# datasets = DatasetDict({"train": jnlpba[0], "validation": jnlpba[1]})


class JnlpbDataset(Dataset):

    def __init__(self, dataset):
        self.dataset = dataset
        self.convert()

    def map_tags(self, row):
        mapping = {
            0: "O",
            1: "B-DNA",
            2: "I-DNA",
            3: "B-RNA",
            4: "I-RNA",
            5: "B-cell_line",
            6: "I-cell_line",
            7: "B-cell_type",
            8: "I-cell_type",
            9: "B-protein",
            10: "I-protein"
        }
        row['ner_tags'] = [[mapping[tag] for tag in row['ner_tags']]][0]
        return row

    def convert(self):
        data = self.dataset
        df_train = pd.DataFrame(data[0])
        df_val = pd.DataFrame(data[1])

        # todo DRY
        df_train = df_train.apply(self.map_tags, axis=1)
        l = []
        l_temp = []
        for i in range(len(df_train)):
            for j in range(len(df_train['ner_tags'][i])):
                l_temp.append(df_train['ner_tags'][i]
                              [j] + ': ' + df_train['tokens'][i][j])
            l.append(l_temp)
            l_temp = []
        d = {'spans': l}
        df_train = df_train.assign(spans=l)

        # todo check if map tags io
        df_val = df_val.apply(self.map_tags, axis=1)
        l = []
        l_temp = []
        for i in range(len(df_val)):
            for j in range(len(df_val['ner_tags'][i])):
                l_temp.append(df_val['ner_tags'][i][j] +
                              ': ' + df_val['tokens'][i][j])
            l.append(l_temp)
            l_temp = []
        d = {'spans': l}
        df_val = df_val.assign(spans=l)

        train = Dataset.from_pandas(df_train)
        val = Dataset.from_pandas(df_val)
        data = DatasetDict({"train": train, "validation": val})
        return data
    
    


input_dataset = JnlpbDataset(dataset=jnlpba)


Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [81]:
print(input_dataset.convert())

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 10
    })
})


In [63]:
# iter over every row from jnlpba[0] -
# convert ner tags to tag names -
# create a list -
# from row iter over len ner tags -
# create a string of elements from ner_tags "elem + ': ' + elem from tokens -
# add to list dict "spans": [some list] -
# add this list as column to the dataset[train] -
# repeat for val -
# put in class

df_train = pd.DataFrame(jnlpba[0])
df_val = pd.DataFrame(jnlpba[1])

mapping = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-RNA",
    4: "I-RNA",
    5: "B-cell_line",
    6: "I-cell_line",
    7: "B-cell_type",
    8: "I-cell_type",
    9: "B-protein",
    10: "I-protein"
}


def map_tags(row):
    row['ner_tags'] = [[mapping[tag] for tag in row['ner_tags']]][0]
    return row


df_train = df_train.apply(map_tags, axis=1)
l = []
l_temp = []
for i in range(len(df_train)):
    for j in range(len(df_train['ner_tags'][i])):
        l_temp.append(df_train['ner_tags'][i][j] +
                      ': ' + df_train['tokens'][i][j])
    l.append(l_temp)
    l_temp = []
d = {'spans': l}
df_train = df_train.assign(spans=l)

df_val = df_val.apply(map_tags, axis=1)
l = []
l_temp = []
for i in range(len(df_val)):
    for j in range(len(df_val['ner_tags'][i])):
        l_temp.append(df_val['ner_tags'][i][j] + ': ' + df_val['tokens'][i][j])
    l.append(l_temp)
    l_temp = []
d = {'spans': l}
df_val = df_val.assign(spans=l)

train = Dataset.from_pandas(df_train)
val = Dataset.from_pandas(df_val)
data = DatasetDict({"train": train, "validation": val})


In [64]:
print(data)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'spans'],
        num_rows: 10
    })
})


In [66]:
print(df_train)


   id                                             tokens  \
0   1  [IL-2, gene, expression, and, NF-kappa, B, act...   
1   2  [Activation, of, the, CD28, surface, receptor,...   
2   3  [In, primary, T, lymphocytes, we, show, that, ...   
3   4  [Delineation, of, the, CD28, signaling, cascad...   
4   5  [Our, data, suggest, that, lipoxygenase, metab...   
5   6  [These, findings, should, be, useful, for, the...   
6   7  [The, peri-kappa, B, site, mediates, human, im...   
7   8  [Human, immunodeficiency, virus, type, 2, (, H...   
8   9  [HIV-1, and, HIV-2, display, significant, diff...   
9  10  [Consistent, with, these, differences, ,, we, ...   

                                            ner_tags  \
0  [B-DNA, I-DNA, O, O, B-protein, I-protein, O, ...   
1  [O, O, O, B-protein, I-protein, I-protein, O, ...   
2  [O, B-cell_type, I-cell_type, I-cell_type, O, ...   
3  [O, O, O, B-protein, O, O, O, O, O, O, B-prote...   
4  [O, O, O, O, B-protein, I-protein, O, O, O, O,...   
5  